# Чтение данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
import json

In [3]:
import warnings
import warnings
warnings.filterwarnings('ignore', category=Warning)

In [4]:
tags = pd.read_csv('data/tags.csv', sep=',#', names=['tag', 'name', 'type'])
tags = tags.loc[tags['type'] == '"1"']
tags.head(10)

,tag,name,type
151,"""179""","""О войне""","""1"""
152,"""180""","""военный""","""1"""
153,"""181""","""заграничная поездка""","""1"""
154,"""182""","""блокада""","""1"""
155,"""183""","""тыл""","""1"""
156,"""184""","""фронтовой""","""1"""
157,"""185""","""депортация""","""1"""
158,"""186""","""детский""","""1"""
159,"""187""","""профессиональный""","""1"""
160,"""188""","""путешествие""","""1"""


In [5]:
# список тем типа 1
themes = list(tags['tag'])

In [6]:
diaries = pd.read_csv('data/diary.csv', sep=',#', header=None)
diaries.head()

,0,1,2,3,4,5,6,7,8,9,10
0,"""3""","""4""","""published""","""329""","""1932-09-28""","""1937-04-30""",NaN,"""0""","""0""","""0""","""1425446154"""
1,"""4""","""5""","""published""","""356""","""1942-09-09""","""1944-01-27""",NaN,"""0""","""0""","""1474033861""","""1425446228"""
2,"""5""","""6""","""published""","""28""","""1943-01-31""","""1951-07-03""",NaN,"""0""","""0""","""0""","""1425446370"""
3,"""6""","""7""","""published""","""65""","""0000-00-00""","""0000-00-00""",NaN,"""0""","""0""","""0""","""1425446645"""
4,"""8""","""11""","""published""","""804""","""1943-08-04""","""1961-12-31""",NaN,"""0""","""0""","""0""","""1425487703"""


In [7]:
tags_diaries = pd.read_csv('data/tags_diaries.csv', sep=',#', names=['diary', 'tag'])
tags_diaries.head()

,diary,tag
0,"""3""","""181"""
1,"""3""","""328"""
2,"""4""","""180"""
3,"""4""","""184"""
4,"""8""","""187"""


In [8]:
# датафрейм с id дневников, которым приписана хотя бы одна тема
diaries_with_themes = tags_diaries[tags_diaries['tag'].isin(themes)]

In [9]:
with open('data/notes.json', 'r') as f:
        notes = json.load(f)

In [10]:
rows = []

for note in notes[2]['data']:
    rows.append([note['id'], note['text'], note['diary']])

In [11]:
notes = pd.DataFrame(rows, columns=['id', 'text', 'diary'])

In [12]:
# датафрейм с id и текстом записей и id их дневника
notes.head()

,id,text,diary
0,39081,Барабинск. Татарка.,130
1,39082,Омск. Утро. Стоим до вечера. Обед. Сухой паек.,130
2,39083,В 6 часов утра еще темно. Станция Называевская...,130
3,39084,Утром рано приехали в Тюкалинск. Из автоотряда...,130
4,39085,"Так давно я не прикасался к дневнику, что он м...",234


In [13]:
notes_without_themes = notes[~notes['diary'].isin(diaries_with_themes['diary'].apply(lambda x: x.strip('"')))]

# Предобработка

In [14]:
from stop_words import get_stop_words
import nltk
from pymystem3 import Mystem
import re

In [15]:
random_seed = 42

In [16]:
stop_words = get_stop_words('ru')
stop_words.append('свой')

In [17]:
m = Mystem()

In [18]:
def tokenize(text):
    comp = re.compile('[А-ЯЁа-яё\-]+')
    tokens = comp.findall(text.lower())
    return tokens

In [19]:
def lemmatize(text):
    return [m.lemmatize(word)[0] for word in text]

In [20]:
# токенизируем заметки
notes_lemma = pd.DataFrame()
notes_lemma['text'] = notes_without_themes.text[:10000].apply(lambda x: tokenize(x))

In [21]:
# лемматизируем заметки
notes_lemma['text'] = notes_lemma.text.apply(lambda x: lemmatize(x))

In [22]:
notes_lemma = notes_lemma.text.apply(lambda x: ' '.join(x))

In [23]:
notes_lemma.head(10)

4      так давно я не прикасаться к дневник что он я ...
251    октябрь в день начинать процесс над демонстран...
256    отныне мы быть снимать шапка проходить мимо ло...
257    второй половина август быть столь горестный и ...
258    конец июль начало август весь запад наш страна...
259    несомненно правительство быть ясно что чехосло...
260    а между то снова доходить сведение о неэффекти...
261    со второй половина май в газета явственно отра...
262    в правда опубликовывать два разный статья но о...
263    всего несколько день проходить после окончание...
Name: text, dtype: object

# K-means and visualization

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(notes_lemma)

tfidf_matrix.shape

(10000, 32774)

In [26]:
# list of the features used in the tf-idf matrix
terms = tfidf_vectorizer.get_feature_names()

In [27]:
from sklearn.cluster import KMeans

In [28]:
num_clusters = 10

km = KMeans(n_clusters=num_clusters)

km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [29]:
from sklearn.externals import joblib

In [30]:
# pickle the model
_ = joblib.dump(km,  'doc_cluster.pkl')

In [31]:
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [51]:
prozhito = {'diary': list(notes_without_themes['diary'][:10000]), 'text': notes_lemma, 'cluster': clusters}

frame = pd.DataFrame(prozhito, index=[clusters] , columns=['diary', 'cluster'])

In [55]:
frame['cluster'].value_counts()

5    3566
4    1496
0    1208
9     811
1     700
6     694
8     595
3     458
7     315
2     157
Name: cluster, dtype: int64

In [56]:
clustered_diaries = {}

In [57]:
print("Top terms per cluster:")
print()
# sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :20]: 
        print(' %s' % terms[ind], end='')
    print() 
    print() 
    
    clustered_diaries[i] = set(frame.loc[i]['diary'].values.tolist())
    
#     print("Cluster %d diaries:\n" % i, end='')
#     for title in frame.loc[i]['id'].values.tolist():
#         print(' %s,' % title, end='\n')
    print()
    print()

Top terms per cluster:

Cluster 0 words: ребенок говорить знать жить видеть становиться яковлев дело давать душа мир друг самый думать бог идти писать любить вчера земля



Cluster 1 words: пьеса театр вечер дмитриев мхат роман оля говорить мелик мольер яков звонить вильямс либретто калужский пойти приходить позвонить звонок большой



Cluster 2 words: миша пьеса вечер калишьян звонок виленкин театр борис приходить кихот прийти большой пойти разговор дон говорить рашель позвонить звонить эрдман



Cluster 3 words: аля час алена обед веранда копель море дома завтрак вечер фира сидеть копа солнце али небо сон катя матушка записывать



Cluster 4 words: россия русский большевик москва советский власть народ правительство дело говорить немец война газета новый мир совет революция рабочий известие ленин



Cluster 5 words: вечер москва вчера говорить час ночь писать идти утро получать письмо дело читать становиться работа давать сделать приходить хороший ехать



Cluster 6 words: немец маши

In [60]:
for cluster in clustered_diaries:
    print('Cluster %d:' %cluster)
    print(clustered_diaries[cluster])
    print()

Cluster 0:
{'178', '32', '241', '52', '193', '10', '236', '68', '234', '39', '15', '693', '131', '40', '36', '836', '2', '30', '49', '75', '64', '18', '242', '169', '5', '17', '44', '235', '37', '60', '33', '13', '43', '20'}

Cluster 1:
{'37', '178', '32', '193', '18', '10', '242', '44', '15'}

Cluster 2:
{'2', '32', '193', '18', '13', '169', '15'}

Cluster 3:
{'169', '10'}

Cluster 4:
{'178', '32', '241', '52', '10', '129', '39', '15', '131', '40', '30', '49', '75', '64', '48', '242', '44', '17', '76', '235', '60', '33', '13', '43'}

Cluster 5:
{'178', '9', '32', '241', '52', '193', '10', '129', '236', '68', '39', '15', '55', '693', '131', '40', '36', '836', '2', '30', '49', '75', '64', '18', '48', '242', '169', '5', '17', '44', '76', '235', '37', '60', '33', '13', '43', '20'}

Cluster 6:
{'55', '37', '178', '836', '32', '52', '49', '33', '18', '13', '10', '242', '236', '40', '48', '44', '15'}

Cluster 7:
{'52', '30', '64', '193', '33', '18', '10', '48', '242', '68', '15'}

Cluster 8: